# IMAGE PROCESSING TOOLBOX 


Master in Computer Vision
University of Burgundy 
Danie Jianah SONIZARA 

In [18]:
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
import tkinter
import tkinter.messagebox
import tkinter.filedialog
from tkinter.filedialog import *
from tkinter import *
from tkinter import Label,Tk
from PIL import Image, ImageTk
import glob
%matplotlib inline

##### 1. Load an Display 

In [19]:
# Load an image 
def LoadImage():
    filename = filedialog.askopenfilename()
    img_color = cv2.imread(filename)
   
    # display the image with OpenCV imshow()
    cv2.imshow('Original Image', img_color)
    


    # The OpenCV waitKey() function is a required keyboard binding 
    # function after imwshow()
#     cv2.waitKey(0)

#     # destroy all windows command
#     cv2.destroyAllWindows()
    global image
    image = img_color
    
# Load a video
def LoadVideo():
    filename = filedialog.askopenfilename()
    video = cv2.VideoCapture(filename)
    while(video.isOpened()):
        ret, frame = video.read()
        cv2.imshow('Video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            video.release()
            cv2.destroyAllWindows()
            
# Camera Live 
def LoadCamera(mirror=True):
    cam = cv2.VideoCapture(0)
    while True:
        ret_val, img = cam.read()
        if mirror: 
            img = cv2.flip(img, 1)
            cv2.imshow('my webcam', img)
            if cv2.waitKey(1) == 27: 
                break  # esc to quit
                cv2.destroyAllWindows()

##### 2. Add noise to the original image 

In [20]:
#  a. Add salt and pepper
# Replaces random pixels with 0 or 1.
def SaltPepperNoise():
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.004
    out = image
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
    out[coords] = 1
    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
    out[coords] = 0
    
    cv2.imshow('Salt and Pepper noise', out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def GaussianNoise():
    row,col,ch= image.shape
    mean = 0
    #var = 0.1
    #sigma = var**0.5
    gauss = np.random.normal(mean,1,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    
    cv2.imshow('Gaussian noise  ', noisy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def PoissonNoise():
    vals = len(np.unique(image))
    vals = 2 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    
    cv2.imshow('Poisson noise  ', noisy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
        
def SpeckleNoise():
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)        
    noisy =  image * gauss
    
    cv2.imshow('Speckle noise  ', noisy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    


##### b. Show a logo at a user-defined RegionOf Interest (ROI) of the input 

In [ ]:
def ChooseLogo():
    filename = filedialog.askopenfilename()
    img2 = cv2.imread(filename)
    cv2.imshow('logo',img2)
    global logo
    logo = img2
def SelectROI():
#     # I want to put logo on top-left corner, So I create a ROI
#     r = cv2.selectROI(image)
#     roi = image[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
#     cv2.imshow('Select ROI',roi) # Display cropped image

#     thumbnail = cv2.CreateMat(roi.rows, roi.cols, cv.CV_8UC3)
    # I want to put logo on top-left corner, So I create a ROI
    rows,cols,channels = logo.shape
    roi = image[0:rows, 0:cols ]
    # Now create a mask of logo and create its inverse mask also
    img2gray = cv2.cvtColor(logo,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)
    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(logo,logo,mask = mask)
    # Put logo in ROI and modify the main image
    dst = cv2.add(img1_bg,img2_fg)
    image[0:rows, 0:cols ] = dst
    cv2.imshow('result',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
  

#### c. Convert to a new colorspace 

In [ ]:
# RGB to gray
def RGBtoGray():
    grey = cv2.cvtColor( image, cv2.COLOR_RGB2GRAY )
    cv2.imshow("RGB to Gray image", grey)
    global img_grey
    img_grey = grey 

# RGB to YCR
def RGBtoYCRCB():
    YCRCB = cv2.cvtColor(image, cv2.COLOR_BGR2YCR_CB)
    cv2.imshow("RGB to YCRCB image", YCRCB)

#### d. Compute the histogram of the image 

In [ ]:
def ComputeHistogram():
    hist = cv2.calcHist([image],[0],None,[256],[0,256])
    cv2.imshow("Histogram of the image", hist)
#     # Another way to compute histogram using plot
#     plt.hist(image.ravel(),256,[0,256])
#     plt.title('Histogram ofthe image')
#     plt.plot()
#     plt.show()
    plt.plot()
    plt.xlim([0,256])
    plt.show()

    while True:
        k = cv2.waitKey(0) & 0xFF     
        if k == 27: break             # ESC key to exit 
        cv2.destroyAllWindows()

#### f. Morphological operators

In [ ]:
kernel = np.ones((5,5),np.uint8)
global kernel
# Dilate
def Dilate():
    dilation = cv2.dilate(image,kernel,iterations = 1)
    cv2.imshow("Dilation", dilation)
# Erode 
def Erode():
    erosion = cv2.erode(image,kernel,iterations = 1)
    cv2.imshow("Erosion", erosion)
# Open 
def Open():
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    cv2.imshow("Opening", opening)
# Close
def Close():
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    cv2.imshow("Close", closing)


#### g. Blur

In [ ]:
# Averaging
def Averaging():
    averaging = cv2.blur(image,(5,5))
    cv2.imshow("Averaging Blur", averaging)
# Gaussian Blurring
def GaussianBlur():
    gaussianBlur = cv2.GaussianBlur(image,(5,5),0)
    cv2.imshow("Gaussian Blurring", gaussianBlur)
# Median Blurring
def MedianBlur():
    median = cv2.medianBlur(image,5)
    cv2.imshow("Median Blurring", median)

#### h.i. Edge detector operators

In [ ]:
# Sobel operator

def SobelOp():
    sobelx = cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5)
    cv2.imshow("Sobel operator", sobelx)
# Laplacian operator
def LaplacianOp():
    laplacian = cv2.Laplacian(image,cv2.CV_64F)
    cv2.imshow("Laplacian operator", laplacian)
# Canny edge detction
def Canny():
    canny = cv2.Canny(image,100,200)
    cv2.imshow("Canny edge detection", canny)

#### j. Extract lines and circles using Hough transform.

In [ ]:
# Extract lines 
def HoughLine():
    img= image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray,100,200,apertureSize = 3)

    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
    for x in range(0, len(lines)):
        for x1,y1,x2,y2 in lines[x]:
            img= cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
        
    cv2.imshow("Extracting Lines using Hough Transform ", img)
    cv2.waitKey(0)

    

        #cv2.imwrite('houghlines3.jpg',image)
            
# Extract circles using Hough transform
def HoughCircle():
    img = cv2.medianBlur(image,5)
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=50,param2=30,minRadius=0,maxRadius=0)

    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # draw the outer circle
        cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

    cv2.imshow('detected circles',cimg)
    cv2.waitKey(0)
    
    

#### k.Find contours of connected objects and draw them

In [ ]:
def FindDrawContour ():
    img = image
    imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,127,255,0)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # Draw the contours
    im = cv2.drawContours(img, contours, -1, (0,255,0), 3)
    
    cv2.imshow('Find and Draw Contour on image',im)
    cv2.waitKey(0)
    

#### l. Shape descriptors (bounding box, minimum en-closing circle)

In [ ]:
# Contour Area 
def ContourArea():
    grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    ret,thresh = cv2.threshold(grey,127,255,0)
    _, contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    M = cv2.moments(cnt)
    
    area = cv2.contourArea(cnt) 
    im = cv2.drawContours(img, area, -1, (0,255,0), 3)
    cv2.imshow('Find Contour Area of an image',im)
    cv2.waitKey(0)
# Contour perimeter 
def ContourPerimeter():
    grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    ret,thresh = cv2.threshold(image,127,255,0)
    _, contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    M = cv2.moments(cnt)
    
    perimeter = cv2.arcLength(cnt,True)
    cv2.imshow('Find and Draw Contour perimeter on image',perimeter)
    cv2.waitKey(0)

# Contour Approximation
def ContourApproximation():
    grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    ret,thresh = cv2.threshold(grey ,127,255,0)
    _, contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    M = cv2.moments(cnt)
    
    epsilon = 0.1*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    cv2.imshow('Find and Draw Contour Approximation of an image',approx)
    cv2.waitKey(0)
                 
#  Straight Bounding Rectangle
def BoundingRectangle():
    grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    ret,thresh = cv2.threshold(grey ,127,255,0)
    _, contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    M = cv2.moments(cnt)
                                 
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.imshow('Find and Draw Bounding Rectangle an image',grey )
    cv2.waitKey(0)
                                 
# Applying minimum en-closing circle
def MinEnclosingCircle():
    grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    ret,thresh = cv2.threshold(grey ,127,255,0)
    _, contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    M = cv2.moments(cnt)
                                 
    (x,y),radius = cv2.minEnclosingCircle(cnt)
    center = (int(x),int(y))
    radius = int(radius)
    minCircle = cv2.circle(img,center,radius,(0,255,0),2)
    cv2.imshow('Find and Draw minimum en-closing circle an image',minCircle )
    cv2.waitKey(0)

#### m. Harris Corner extraction 

In [ ]:
# Extract corners using Harris and apply non-maximal suppression.
def HarrisCorner ():
    img = image;
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)

    #result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # Threshold for an optimal value, it may vary depending on the image.
    img[dst>0.01*dst.max()]=[0,0,255]

    cv2.imshow('Harris Corner Detection',img)
    
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()


#### n. Features  Extraction  

In [ ]:
# Extract FAST 
def FASTextract():
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Initiate FAST object with default values
    fast = cv2.FastFeatureDetector()
    # find and draw the keypoints
    kp = fast.detect(img,None)
    img2 = cv2.drawKeypoints(img, kp, color=(255,0,0))
    # Print all default params
    print ("Threshold: ", fast.getInt('threshold'))
    print ("nonmaxSuppression: ", fast.getBool('nonmaxSuppression'))
    print ("neighborhood: ", fast.getInt('type'))
    print ("Total Keypoints with nonmaxSuppression: ", len(kp))
    cv2.imshow('FAST Extraction wihnon maximal Suppression',img2)

    # Disable nonmaxSuppression
    fast.setBool('nonmaxSuppression',0)
    kp1 = fast.detect(img,None)

    print ("Total Keypoints without nonmaxSuppression: ", len(kp1))

    img3 = cv2.drawKeypoints(img, kp1, color=(255,0,0))
    cv2.imshow('FAST Extraction without non maximal Suppression',img3)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

 # Extract SURF
def SURFextract(): 
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    surf = cv2.SURF(400)
    # Find keypoints and descriptors directly
    kp, des = surf.detectAndCompute(img,None)
    # Check present Hessian threshold
    print (surf.hessianThreshold)
    # We set it to some 50000. Remember, it is just for representing in picture.
    # In actual cases, it is better to have a value 300-500
    surf.hessianThreshold = 50000
    # Again compute keypoints and check its number.
    kp, des = surf.detectAndCompute(img,None)
    img2 = cv2.drawKeypoints(img,kp,None,(255,0,0),4)
    cv2.imshow('SURF Extraction wihnon maximal Suppression',img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
# Extract SIFT 
def SIFTextract():
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    img=cv2.drawKeypoints(gray,kp)
    cv2.imshow('SIFT Extraction wihnon maximal Suppression',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### GUI

In [ ]:
# Main window
MyToolboxWindow = Tk()
MyToolboxWindow.title("Image Processing ToolBox")

# Create a menu toolbar
menubar = Menu(MyToolboxWindow)

# Sow the menu toolbar
MyToolboxWindow.config(menu=menubar)

# *********************************************
# File command 
fileMenu = Menu(menubar,tearoff=0)
menubar.add_cascade(label="File", menu=fileMenu)

# Load 
LoadMenu = Menu(fileMenu,tearoff=0)
fileMenu.add_cascade(label="Load", menu=LoadMenu)
# Load image 
LoadMenu.add_command(label="Image", command=LoadImage)
# Load Video
LoadMenu.add_command(label="Video", command=LoadVideo)
# My webcam camera live Video
LoadMenu.add_command(label="Webcam/Camera", command=LoadCamera)


# Quit 
fileMenu.add_command(label="Quit",command=MyToolboxWindow.destroy)

# **********************************************
# Adding logo
LogoMenu = Menu(menubar,tearoff=0)
menubar.add_cascade(label="ROI", menu=LogoMenu )
# ChooseLogo
LogoMenu.add_command(label="Choose Logo",command=ChooseLogo)
# Select ROI and Display ROI
LogoMenu.add_command(label="Add logo",command=SelectROI)
# **********************************************
# Noise
NoiseMenu = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Noise", menu=NoiseMenu)
# Salt and pepper noise
NoiseMenu.add_command(label="Salt&Pepper",command=SaltPepperNoise)
# Gaussian noise
NoiseMenu.add_command(label="Gaussian",command=GaussianNoise)
# Poisson noise 
NoiseMenu.add_command(label="Poisson",command=PoissonNoise)
# Speckle noise
NoiseMenu.add_command(label="Speckle",command=SpeckleNoise)

# **********************************************
# Changing colorspace 
ColorspaceMenu = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Colorspace", menu=ColorspaceMenu)
# RGB to Gray
ColorspaceMenu.add_command(label="RGBtoGray",command=RGBtoGray)
# RGB to YCRCB
ColorspaceMenu.add_command(label="RGBtoYCRCB",command=RGBtoYCRCB)

# **********************************************
# Histogram
HistogramMenu = Menu(menubar,tearoff=0)
menubar.add_cascade(label=" Histogram", menu=HistogramMenu)
# Compute Histogram
HistogramMenu.add_command(label="Compute Histogram",command=ComputeHistogram)
# Histogram Equalization 
HistogramMenu.add_command(label="Histogram Equalization",command=ComputeHistogram)

# **********************************************
# Morphological operators
Morphological = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Morphological", menu=Morphological)
# Dilate
Morphological.add_command(label="Dilate",command=Dilate)
# Erode 
Morphological.add_command(label="Erode",command=Erode)
# Open 
Morphological.add_command(label="Open",command=Open)
# Close
Morphological.add_command(label="Close",command=Close)

# **********************************************
# Bluring
Blur = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Blur", menu=Blur )
# Averaging
Blur.add_command(label="Averaging",command=Averaging)
# Gaussian Blurring 
Blur.add_command(label="Gaussian Blurring",command=GaussianBlur)
# Median Blurring
Blur.add_command(label="Median Blurring",command=MedianBlur)

# **********************************************
# Edge detectors 
Edges= Menu(menubar,tearoff=0)
menubar.add_cascade(label="Edge", menu=Edges )
# Sobel operator
Edges.add_command(label="Sobel operator",command=SobelOp)
# Laplacian operator
Edges.add_command(label="Laplacian operator",command=LaplacianOp)
# Canny edge detction
Edges.add_command(label="Canny edge detction",command=Canny)

# **********************************************
# Hough transform
Hough= Menu(menubar,tearoff=0)
menubar.add_cascade(label="Hough Transform ", menu=Hough)
# Lines Extraction
Hough.add_command(label="Line extraction using Hough Transform",command=HoughLine)
# Circles Extraction 
Hough.add_command(label="Circle extraction using Hough Transform",command=HoughCircle)
# **********************************************
# Shape descriptors
ShapeDescriptors= Menu(menubar,tearoff=0)
menubar.add_cascade(label="Shape Descriptors ", menu=ShapeDescriptors)
# Find contours of connected objects and draw them
ShapeDescriptors.add_command(label="Connected object Contours",command=FindDrawContour)
# Contour Area
ShapeDescriptors.add_command(label="Contour Area ",command=ContourArea)
# Contour Perimeter
ShapeDescriptors.add_command(label="Contour perimeter",command=ContourPerimeter)
# Contour Approximation
ShapeDescriptors.add_command(label="Contour Approximation ",command=ContourApproximation)
# BoundingRectangle
ShapeDescriptors.add_command(label="Bounding Rectangle",command=BoundingRectangle)
# minimum en-closing circle
ShapeDescriptors.add_command(label="Minimum en-closing circle ",command=MinEnclosingCircle)

# **********************************************
# Corner detection
Features = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Features  ", menu=Features)
# Extract corners using Harris and apply non-maximal suppression.
Features.add_command(label="Harris Cornner detection ",command=HarrisCorner )
# Extract FAST
Features.add_command(label="FAST Extraction ",command= FASTextract )
# Extract SURF.
Features.add_command(label="SURF Extraction ",command= SURFextract )
# Extract SIFT.
Features.add_command(label="SIFT Extraction ",command= SIFTextract )
# **********************************************
# Feature matching 
Matching = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Matching ", menu=Matching)

# **********************************************
# Epipolar geometry
EpipolarGeometry = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Epipolar Geometry  ", menu=EpipolarGeometry)
# **********************************************
# Face Detection
FaceDetection = Menu(menubar,tearoff=0)
menubar.add_cascade(label="Face Detection ", menu=FaceDetection)

MyToolboxWindow.mainloop()

In [ ]:
# Face Detection
# Using  Deep Learning
# Using Cascading classifiers


